<a href="https://colab.research.google.com/github/kim1987/aiffel/blob/main/aiffel/exploration/ex_onetwo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
!cp /content/drive/MyDrive/colabdata/aiffel/ex_onetwo/data.tar /content

In [45]:
!tar xvf data.tar

./data/
./data/yoochoose-clicks.dat
./data/dataset-README.txt
./data/ratings.dat
./data/users.dat
./data/movies.dat
./data/README
./data/yoochoose-test.dat
./data/yoochoose-buys.dat


In [46]:
import os

In [47]:
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [88]:

data_path = '/content/data/'
train_path = data_path + 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [89]:
data['Time'] = data['Time'].apply(pd.Timestamp,unit='s')
data

,UserId,ItemId,Rating,Time
31,1,3186,4,2000-12-31 22:00:19
22,1,1270,5,2000-12-31 22:00:55
27,1,1721,4,2000-12-31 22:00:55
37,1,1022,5,2000-12-31 22:00:55
24,1,2340,3,2000-12-31 22:01:43
...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29
999988,6040,1921,4,2001-08-10 14:41:04
1000172,6040,1784,3,2001-08-10 14:41:04
1000167,6040,161,3,2001-08-10 14:41:26


In [90]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

2000-04-25 23:05:32
2003-02-28 17:49:50


In [91]:
sep_month = latest - pd.Timedelta(days=500)
data = data[data['Time']>= sep_month]
data

,UserId,ItemId,Rating,Time
2503,20,1694,3,2001-12-29 23:37:51
2512,20,1468,3,2001-12-29 23:37:51
2513,20,3717,2,2001-12-29 23:37:51
2517,20,2858,4,2001-12-29 23:37:51
2504,20,2641,4,2001-12-29 23:38:35
...,...,...,...,...
994100,6002,2013,4,2002-02-24 04:24:39
993890,6002,2520,4,2002-02-24 04:24:40
994045,6002,1387,5,2002-02-24 04:25:20
993900,6002,1927,4,2002-02-24 04:25:58


In [93]:
data['tdiffer'] = data.groupby('UserId')['Time'].diff().fillna(pd.Timedelta(0))

In [94]:
seconds = data[data['tdiffer']!=pd.Timedelta(0)]['tdiffer'].iloc[:].apply(lambda x : x.total_seconds()).mean()
seconds = pd.Timedelta(seconds=seconds)

In [95]:
data['SessionId']=0
max_value = data['UserId'].nunique()*10

In [96]:
from tqdm import tqdm
session_number = max_value
session_change=0
for idx, data_series in tqdm(enumerate(data.iterrows())):
  if idx >0:
    if data_series[1]['UserId'] != before_user_id:
      session_number+=1
    elif data_series[1]['tdiffer']>seconds:
      session_number+=1
  data.iat[idx,5]=session_number
  before_user_id=data_series[1]['UserId']

34823it [00:07, 4836.89it/s]


In [97]:
data
data_copy = data.copy()

In [98]:
data = data_copy.copy()

In [99]:
def cleanse_recursive(data: pd.DataFrame, shortest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('SessionId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['SessionId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [100]:
data = cleanse_recursive(data, shortest=2, least_click=5)
data

,UserId,ItemId,Rating,Time,tdiffer,SessionId
2503,20,1694,3,2001-12-29 23:37:51,0 days 00:00:00,6640
2512,20,1468,3,2001-12-29 23:37:51,0 days 00:00:00,6640
2513,20,3717,2,2001-12-29 23:37:51,0 days 00:00:00,6640
2517,20,2858,4,2001-12-29 23:37:51,0 days 00:00:00,6640
2504,20,2641,4,2001-12-29 23:38:35,0 days 00:00:44,6640
...,...,...,...,...,...,...
994100,6002,2013,4,2002-02-24 04:24:39,0 days 00:00:26,9920
993890,6002,2520,4,2002-02-24 04:24:40,0 days 00:00:01,9920
994045,6002,1387,5,2002-02-24 04:25:20,0 days 00:00:40,9920
993900,6002,1927,4,2002-02-24 04:25:58,0 days 00:00:38,9920


In [101]:
data = data[['SessionId','ItemId','Time']]

In [103]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('SessionId')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - pd.Timedelta(days=n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - pd.Timedelta(days=n_days)].index

    before_date = data[data['SessionId'].isin(session_in_train)]
    after_date = data[data['SessionId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

In [105]:
tr, test = split_by_date(data, n_days=60)
tr, val = split_by_date(tr, n_days=60)

In [106]:
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["SessionId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [107]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 25185
	 Sessions: 1655
	 Items: 1942
	 First Time : 2001-10-16 17:55:33
	 Last Time : 2002-10-29 23:24:51

* valid Set Stats Info
	 Events: 2797
	 Sessions: 178
	 Items: 1309
	 First Time : 2002-10-14 19:25:37
	 Last Time : 2002-12-30 02:26:14

* test Set Stats Info
	 Events: 3067
	 Sessions: 187
	 Items: 1358
	 First Time : 2002-12-20 03:36:18
	 Last Time : 2003-02-28 17:49:50



In [108]:
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [109]:
tr.sort_values(['SessionId', 'Time'], inplace=True)

In [110]:
save_path = data_path + 'processed'
Path(save_path).mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path + 'train.pkl')
val.to_pickle(save_path + 'valid.pkl')
test.to_pickle(save_path + 'test.pkl')

In [68]:
'''
import pickle
import pandas as pd

tr = pd.read_pickle(save_path / 'train.pkl')
val = pd.read_pickle(save_path / 'valid.pkl')
test = pd.read_pickle(save_path / 'test.pkl')
'''

"\nimport pickle\nimport pandas as pd\n\ntr = pd.read_pickle(save_path / 'train.pkl')\nval = pd.read_pickle(save_path / 'valid.pkl')\ntest = pd.read_pickle(save_path / 'test.pkl')\n"

In [111]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['SessionId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['SessionId'].nunique() + 1, dtype=np.int64)
        offsets[1:] = self.df.groupby('SessionId').size().cumsum()
        return offsets

In [112]:
print(len(tr.groupby('SessionId').size().cumsum()))
print(tr['SessionId'].nunique())

1655
1655


In [113]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=64):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [114]:
len(tr['SessionId'])

25185

In [115]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [116]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [117]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [118]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['SessionId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=32, hsz=50, drop_rate=0.1, lr=0.001, epochs=100, k=21)

In [119]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['SessionId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

       # val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        #print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        #print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['SessionId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [122]:
with tf.device("/device:GPU:0"):
  #model = create_model(args)
  train_model(model, args)



Train:  95%|█████████▍| 698/735 [00:11<00:00, 60.38it/s, accuracy=0.688, train_loss=1.1]


In [123]:

def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 21)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")
    
with tf.device('/device:GPU:0'):
  test_model(model, args, test)


Evaluation:  40%|████      | 36/90 [00:13<00:19,  2.72it/s]

	 - Recall@21: 0.051215
	 - MRR@21: 0.018374



In [ ]:
'''최동현님 아이디어--------------------------'''

In [ ]:
'''
from multiprocessing import Pool
def get_time_differ(UserId):

  differs=0
  end_idx = 0
  for idx,time in enumerate(data[data['UserId']==UserId]['Time'],start=1):
    if idx > 1:
      differs=time-before_time
    else:
      before_time=time
    end_idx = idx
  return differs,end_idx
time_differs = Pool(os.cpu_count()).map(get_time_differ,data['UserId'].unique())
'''

In [ ]:
'''
from multiprocessing import Pool
def get_time_differ(UserId):

  differs = data[data['UserId']==UserId]['Time'].iloc[-1]-data[data['UserId']==UserId]['Time'].iloc[0]
  end_idx = len(data[data['UserId']==UserId]['Time'])
  return differs,end_idx
time_differs = Pool(os.cpu_count()).map(get_time_differ,data['UserId'].unique())
'''#더 오래걸림 

In [ ]:
#data['tdiffer'] = data.groupby('UserId')['Time'].diff().fillna(0.0)

In [ ]:
'''
total_differ = 0
total_num = 0
for differs, nums in time_differs:
  total_differ+=differs
  total_num+=nums
'''

In [ ]:
#avr_time = int(data['tdiffer'].mean())

In [ ]:
#avr_time = total_differ//total_num

In [ ]:
#max_value = len(data)

In [ ]:
#avr_time

In [ ]:
#max(data.groupby('UserId').size())

In [ ]:
'''
def set_session_id(UserId,*,avr_time,siz):

  number_session=0
  user_session = UserId*siz
  for idx, time in enumerate(data[data['UserId']==UserId]['Time']):
    data[data['UserId']==UserId][data['Time']==time]['SessionId']=max_value + user_session + number_session
    if idx > 0:
      if time-before_time >= avr_time:
        number_session+=1
    before_time = time
'''
#쓸모없이 김

In [ ]:
'''
from tqdm import tqdm
session_number = max_value
session_change=0
for idx, data_series in tqdm(enumerate(data.iterrows())):
  user_id = data_series[1]['UserId']
  time = data_series[1]['Time']
  if idx >0:
    if user_id != before_user_id:
      session_number+=1
    elif time-before_time>=avr_time:
      session_number+=1
      session_change+=1
  data.iat[idx,4]=session_number
  before_user_id=user_id
  before_time = time
  '''

In [ ]:
#print(session_change)  

In [ ]:
#data['UserId'].nunique()